# Sentiment Classification


### Generate Word Embeddings and retrieve outputs of each layer with Keras based on Classification task

Word embeddings are a type of word representation that allows words with similar meaning to have a similar representation.

It is a distributed representation for text that is perhaps one of the key breakthroughs for the impressive performance of deep learning methods on challenging natural language processing problems.

We willl use the imdb dataset to learn word embeddings as we train our dataset. This dataset contains 25,000 movie reviews from IMDB, labeled with sentiment (positive or negative). 



### Dataset

`from keras.datasets import imdb`

Dataset of 25,000 movies reviews from IMDB, labeled by sentiment (positive/negative). Reviews have been preprocessed, and each review is encoded as a sequence of word indexes (integers). For convenience, the words are indexed by their frequency in the dataset, meaning the for that has index 1 is the most frequent word. Use the first 20 words from each review to speed up training, using a max vocab size of 10,000.

As a convention, "0" does not stand for a specific word, but instead is used to encode any unknown word.


### Aim

1. Import test and train data  
2. Import the labels ( train and test) 
3. Get the word index and then Create key value pair for word and word_id. (12.5 points)
4. Build a Sequential Model using Keras for Sentiment Classification task. (10 points)
5. Report the Accuracy of the model. (5 points)  
6. Retrive the output of each layer in keras for a given single test sample from the trained model you built. (2.5 points)


#### Usage:

In [4]:
import numpy as np
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# call load_data with allow_pickle implicitly set to true
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=10000)

# restore np.load for future normal usage
np.load = np_load_old

In [7]:
from keras.preprocessing.sequence import pad_sequences
vocab_size = 10000 #vocab size
maxlen = 300  #number of word used from each review

In [8]:
word_index = imdb.get_word_index()

reverse_word_index = dict(
[(value, key) for (key, value) in word_index.items()])

1646592/1641221 [==============================] - 2s 1us/step


In [9]:
from keras.preprocessing.sequence import pad_sequences
vocab_size = 10000 #vocab size
maxlen = 300  #number of word used from each review

In [11]:
#loading dataset as a list of ints
import numpy as np
# save np.load
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=vocab_size)
np.load = np_load_old
#making  all sequences of the same length
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test =  pad_sequences(x_test, maxlen=maxlen)

## Build Keras Embedding Layer Model
We can think of the Embedding layer as a dicionary that maps a index assigned to a word to a word vector. This layer is very flexible and can be used in a few ways:

* The embedding layer can be used at the start of a larger deep learning model. 
* Also we could load pre-train word embeddings into the embedding layer when we create our model.
* Use the embedding layer to train our own word2vec models.

The keras embedding layer doesn't require us to onehot encode our words, instead we have to give each word a unqiue intger number as an id. For the imdb dataset we've loaded this has already been done, but if this wasn't the case we could use sklearn [LabelEncoder](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html).

In [12]:
from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils, to_categorical
import numpy as np

In [13]:
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

In [14]:
x_train = vectorize_sequences(x_train)
x_test = vectorize_sequences(x_test)

In [15]:
y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')
print("y_train ", y_train.shape)
print("y_test ", y_test.shape)

y_train  (25000,)
y_test  (25000,)


In [16]:
x_val = x_train[:10000]
partial_x_train = x_train[10000:]
y_val = y_train[:10000]
partial_y_train = y_train[10000:]

In [17]:
model = models.Sequential()
model.add(layers.Dense(16, kernel_regularizer=regularizers.l1(0.001), activation='relu', input_shape=(10000,)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, kernel_regularizer=regularizers.l1(0.001),activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [18]:
#Compiling the model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

# Runnning the model and saving it to history variable for reporting loss and accuracy 
history = model.fit(partial_x_train, partial_y_train, epochs=20, batch_size=512, validation_data=(x_val, y_val))

Instructions for updating:
Use tf.cast instead.
Train on 15000 samples, validate on 10000 samples
Epoch 1/20
15000/15000 [==============================] - 9s 575us/step - loss: 1.1488 - acc: 0.6117 - val_loss: 0.8012 - val_acc: 0.8113
Epoch 2/20
15000/15000 [==============================] - 4s 249us/step - loss: 0.7853 - acc: 0.6985 - val_loss: 0.7492 - val_acc: 0.7509
Epoch 3/20
15000/15000 [==============================] - 4s 242us/step - loss: 0.7537 - acc: 0.7343 - val_loss: 0.7071 - val_acc: 0.8379
Epoch 4/20
15000/15000 [==============================] - 4s 239us/step - loss: 0.7263 - acc: 0.7670 - val_loss: 0.6796 - val_acc: 0.8172
Epoch 5/20
15000/15000 [==============================] - 4s 238us/step - loss: 0.7035 - acc: 0.7847 - val_loss: 0.6484 - val_acc: 0.8416
Epoch 6/20
15000/15000 [==============================] - 4s 235us/step - loss: 0.6889 - acc: 0.7968 - val_loss: 0.6304 - val_acc: 0.8454
Epoch 7/20
15000/15000 [==============================] - 4s 236us/step - 

In [19]:
results = model.evaluate(x_test, y_test)
print("Test Loss : ", results[0])
print("Test Accuracy :", results[1]*100,'%' )

25000/25000 [==============================] - 4s 167us/step
Test Loss :  0.585085647315979
Test Accuracy : 84.38 %


## Retrive the output of each layer in keras for a given single test sample from the trained model you built

In [20]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 16)                160016    
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 17        
Total params: 160,305
Trainable params: 160,305
Non-trainable params: 0
_________________________________________________________________


In [21]:
from keras import backend as K

inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functor = K.function([inp, K.learning_phase()], outputs )   # evaluation function

In [22]:
layer_outs = functor([x_test[:1], 1.])

for i in range(len(layer_outs)):
  print("The output of Layer ", str(i+1)," is ")
  print(layer_outs[i])

The output of Layer  1  is 
[[0.         0.08964733 0.         0.         0.04054572 0.1782749
  0.         0.0374633  0.         0.         0.         0.
  0.01538302 0.         0.04298142 0.17496365]]
The output of Layer  2  is 
[[0.         0.17929466 0.         0.         0.08109145 0.
  0.         0.07492661 0.         0.         0.         0.
  0.03076605 0.         0.         0.3499273 ]]
The output of Layer  3  is 
[[0.42975658 0.3242338  0.01763772 0.01291595 0.00379942 0.30628127
  0.01036339 0.01070884 0.         0.03620822 0.         0.53964674
  0.48161554 0.02051652 0.01226871 0.04310495]]
The output of Layer  4  is 
[[0.         0.6484676  0.03527545 0.         0.         0.
  0.02072678 0.         0.         0.07241644 0.         1.0792935
  0.         0.04103305 0.         0.08620991]]
The output of Layer  5  is 
[[0.23300138]]
